In [1]:
#export
from k1lib.callbacks import Callback, Callbacks
import k1lib as k1lib

In [2]:
#export
@k1lib.patch(Callback.cls)
class HookParam(Callback, k1lib.ListContainer):
    """Records means and stds of all parameters. Display
    this in your cell for more."""
    def __init__(self):
        super(HookParam, self).__init__()
    def startRun(self):
        if len(self) == 0: # set things up first time only
            self.clear()
            for name, param in self.model.named_parameters():
                self.append(k1lib.Object.fromDict({"name": name, "param": param, "means": [], "stds": []})
                            .withRepr(f"Param `{name}`. Use `.mean` or `.stds` for detailed data"))
    def startBatch(self):
        for data in self:
            data.means.append(data.param.detach().mean().item())
            data.stds.append(data.param.detach().std().item())
    def __repr__(self):
        s = f", {len(self[0].means)} means and stds each" if len(self) > 0 else ""
        names = "\n".join([f"  {i}. {data.name}" for i, data in enumerate(self)])
        return f"""{super()._reprHead}: {len(self)} params{s}:
{names}

Use...
- cb.plot(): to quickly look at everything
- cb[i]: to view a single param
{super()._reprCan}"""
    def plot(self, _slice:slice=slice(None, None, None), logScale=False):
        logScale = "log" if logScale else "linear"
        def _plot(i, objF, title):
            _plt.subplot(1, 2, i)
            for m in self:
                obj = objF(m)
                _plt.plot(range(len(obj))[_slice], obj[_slice])
            _plt.title(title); _plt.yscale(logScale)
        _plt.figure(figsize=(10, 3), dpi=100)
        _plot(1, lambda m: m.means, "Means")
        _plot(2, lambda m: m.stds, "Stds")
        _plt.figlegend([data.name for data in self], loc='right'); _plt.show()
@k1lib.patch(Callbacks)
def withHookParam(self): return self.append(HookParam())

In [3]:
!../../export.py _callbacks/hookParam

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> build/lib/k1lib
copying k1lib/viz.py -> buil